In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [ ]:
train = pd.read_csv("C:\\Users\\Win10\\Downloads\\train- Capillary 4\\train.csv",parse_dates=['OrderDate']).reset_index()
test = pd.read_csv("C:\\Users\\Win10\\Downloads\\test_nFNPSyV.csv")
ss = pd.read_csv("C:\\Users\\Win10\\Downloads\\sample_submission_qfCnaKZ.csv")
pa = pd.read_csv("C:\\Users\\Win10\\Downloads\\train- Capillary 4\\product_attributes.csv")

In [ ]:
pa = pd.pivot_table(pa,columns='attribute_name',index='productid',values='attributevalue').reset_index()
train = pd.merge(train,pa)
global_top = train.groupby("productid").nunique().sort_values('UserId',ascending = False).head(10)

In [ ]:
for t in tqdm(test.iterrows(),total=2350):
    user_id = t[1]['UserId']
    user_top = pd.concat([train[train['UserId'] == user_id].groupby("productid")[['Quantity']].count(),
               train[train['UserId'] == user_id].groupby("productid")[['OrderDate']].max()],
              axis = 1).sort_values(['Quantity','OrderDate'],ascending = [False,False])
    preds = list(user_top.head(10).index.values)

In [1]:
import pandas as pd

raw_data = pd.read_csv("C:\\Users\\Win10\\Downloads\\train.csv")[['UserId','productid','Quantity']]

In [2]:
n_latent_factors_user = 5
n_latent_factors_user = 8

In [3]:
import keras
from keras.optimizers import Adam

n_users = len(raw_data.UserId.unique())
user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)

Using TensorFlow backend.


In [4]:
product_input = keras.layers.Input(shape=[128,],name='Product')
product_vec = keras.layers.Flatten(name='FlattenProduct')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='Product-Embedding')(product_input))
product_vec = keras.layers.Dropout(0.2)(product_vec)

In [5]:
concat = keras.layers.merge([user_vec, product_vec], mode='concat',name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, product_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

C:\Users\Win10\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
C:\Users\Win10\Anaconda3\envs\deeplearning\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [6]:
import numpy as np

product_embeddings = pd.DataFrame(np.load('product_embeddings.npy'))
product_id = np.load('product_id.npy')
product = pd.DataFrame()
product['productid'] = [int(i.split(".")[0]) for i in product_id]
product = product.join(product_embeddings)

In [7]:
train = pd.merge(raw_data, product, on=['productid'])

In [8]:
train[train < 0] = 0

In [11]:
history = model.fit([train.UserId, train.iloc[:,3:].values], train.Quantity, epochs=5, batch_size=128, shuffle=True)

Epoch 1/5
132551/132551 [==============================] - 7s 51us/step - loss: 0.0143
Epoch 2/5
132551/132551 [==============================] - 7s 51us/step - loss: 0.0143
Epoch 3/5
132551/132551 [==============================] - 7s 51us/step - loss: 0.0143
Epoch 4/5
132551/132551 [==============================] - 7s 51us/step - loss: 0.0143
Epoch 5/5
132551/132551 [==============================] - 7s 51us/step - loss: 0.0144


In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Product (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 8)         222232      User[0][0]                       
__________________________________________________________________________________________________
Product-Embedding (Embedding)   (None, 128, 8)       222232      Product[0][0]                    
__________________________________________________________________________________________________
FlattenUse